In [2]:
import numpy as np
import deepxde as dde
import tensorflow as tf

data_x = np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])

data_sigma = np.array([[43584350.6], [43195861.8]])  


X_test=np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])
y_test=np.array([[43584350.6], [43195861.8]]) 

print(data_x)
print(data_sigma)
assert data_x is not None, "data_x is None!"
assert data_sigma is not None, "data_sigma is None!"
print(type(data_x))  # Should print <class 'numpy.ndarray'>
print(type(data_sigma))  # Should print <class 'numpy.ndarray'>



import deepxde as dde

# Use the DataSet data object to create a data-driven task
data = dde.data.dataset.DataSet(X_train=data_x, y_train=data_sigma, X_test=X_test, y_test=y_test)

# Define the network for σ
layer_size = [2] + [50] * 3 + [1]  # 2 inputs (x, y ) -> hidden layers -> 1 output (σ)
activation = "tanh"
initializer = "Glorot uniform"
net_sigma = dde.nn.FNN(layer_size, activation,initializer)

# Define the model
model_sigma = dde.Model(data, net_sigma)

# Compile the model with data
model_sigma.compile("adam", lr=1e-3)

# Train the model
losshistory, train_state = model_sigma.train(epochs=5000)


[[0.014598   0.014112  ]
 [0.014598   0.01610713]]
[[43584350.6]
 [43195861.8]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>



Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
Compiling model...
Building feed-forward neural network...


c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(



'build' took 0.356271 s


'compile' took 0.559310 s

Training model...

Step      Train loss    Test loss     Test metric
0         [1.88e+15]    [1.88e+15]    []  
1000      [1.88e+15]    [1.88e+15]    []  
2000      [1.88e+15]    [1.88e+15]    []  
3000      [1.88e+15]    [1.88e+15]    []  
4000      [1.88e+15]    [1.88e+15]    []  
5000      [1.88e+15]    [1.88e+15]    []  

Best model at step 5000:
  train loss: 1.88e+15
  test loss: 1.88e+15
  test metric: []

'train' took 5.206884 s



In [3]:
def pde_C_L(x, C_L):
    dC_L_dt = dde.grad.jacobian(C_L, x, i=0, j=2)
    dC_L_dx2 = dde.grad.hessian(C_L, x, i=0, j=0)
    dC_L_dy2 = dde.grad.hessian(C_L, x, i=0, j=1)
    eq = dC_L_dt - (1/3.8) * (dC_L_dx2 + dC_L_dy2)
    return eq

In [10]:

t_last = 1.0

# Define the constraint for random points A and B
def random_point_constraint(model, x_A, y_A, x_B, y_B, model_sigma):
    # Get the predictions of sigma for the random points
    sigma_A = model_sigma.predict(np.array([[x_A, y_A]]))
    sigma_B = model_sigma.predict(np.array([[x_B, y_B]]))

    # Define constants for the constraint (e.g., VH, R, T)
    VH, R, T = 1.0, 1.0, 1.0  # Adjust based on your problem
    exponent = (VH / (R * T)) * (sigma_A - sigma_B)

    # Model predictions at the two points
    C_A = model.predict(np.array([[x_A, y_A, t_last]]))  # C(x_A, y_A, t_last)
    C_B = model.predict(np.array([[x_B, y_B, t_last]]))  # C(x_B, y_B, t_last)

    # Return the difference between the constraint equation
    return C_A - C_B * np.exp(exponent)



In [23]:
import tensorflow as tf

def custom_loss(y_true, y_pred, model, data):
    # Compute the standard loss (e.g., mean squared error)
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))

    # Add a custom constraint or penalty (e.g., random point constraint)
    constraint_loss = 0.0
    num_pairs = 10  # Number of random pairs to sample

    for _ in range(num_pairs):
        # Sample random points
        x_A = np.random.uniform(-L, L)
        y_A = np.random.uniform(-L, L)
        x_B = np.random.uniform(-L, L)
        y_B = np.random.uniform(-L, L)

        # Calculate the constraint term
        sigma_A = model_sigma.predict(np.array([[x_A, y_A]]))
        sigma_B = model_sigma.predict(np.array([[x_B, y_B]]))

        VH, R, T = 1.0, 1.0, 1.0  # Adjust constants as needed
        exponent = (VH / (R * T)) * (sigma_A - sigma_B)
        C_A = model.predict(np.array([[x_A, y_A, t_last]]))
        C_B = model.predict(np.array([[x_B, y_B, t_last]]))

        # Constraint term
        constraint_loss += tf.reduce_mean(tf.square(C_A - C_B * np.exp(exponent)))

    # Total loss is the sum of the original loss and constraint loss
    return mse_loss + constraint_loss / num_pairs


In [41]:
L = 1.0

def custom_loss(y_true, y_pred):
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    
    # Add the random point constraint loss
    constraint_loss = 0.0
    num_pairs = 10
    
    for _ in range(num_pairs):
        # Sample random points in the domain
        x_A, y_A = np.random.uniform(-L, L), np.random.uniform(-L, L)
        x_B, y_B = np.random.uniform(-L, L), np.random.uniform(-L, L)
        
        # Compute the constraint
        sigma_A = model_sigma.predict(np.array([[x_A, y_A]]))
        sigma_B = model_sigma.predict(np.array([[x_B, y_B]]))
        
        VH, R, T = 1.0, 1.0, 1.0
        exponent = (VH / (R * T)) * (sigma_A - sigma_B)
        
        # Predictions for C at random points
        C_A = model.predict([[x_A, y_A, t_last]])
        C_B = model.predict([[x_B, y_B, t_last]])
        
        constraint_loss += tf.reduce_mean(tf.square(C_A - C_B * np.exp(exponent)))
    
    return mse_loss + constraint_loss / num_pairs

In [42]:
# Define the square domain (e.g., 2x2 square with side length L)

square = dde.geometry.Rectangle([-L, -L], [L, L])

# Define the circular hole at the center (radius R)
R = 0.1  # Radius of the hole
circle = dde.geometry.Disk([0, 0], R)

# Geometry and time domain
geom = dde.geometry.csg.CSGDifference(square, circle)
timedomain = dde.geometry.TimeDomain(0, t_last)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Define the initial condition: C(x, y, 0) = 20
def initial_condition(x):
    return 20

# Define Dirichlet boundary conditions (e.g., C = 0 on the boundaries)
def boundary_condition(x, on_boundary):
    return on_boundary

# Create the initial and boundary conditions
ic = dde.icbc.IC(geomtime, lambda x: 20, lambda _, on_initial: on_initial)
bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, boundary_condition)

data = dde.data.TimePDE(
    geomtime,
    pde_C_L,[ic],
    num_domain=1000,
    num_boundary=150,
    num_initial=180,
    num_test=1000,
    )

# Network for C_L
layer_size = [3] + [64] * 3 + [1]  
activation = "tanh"
initializer = "Glorot uniform"
net_C_L = dde.nn.FNN(layer_size, activation, initializer)



# Define the model with the PDE, initial and boundary conditions
model= dde.Model(data, net_C_L)

# Compile the model with the custom loss function
model.compile("adam", lr=1e-3, loss=custom_loss)

# Train the model
losshistory, train_state = model.train(epochs=10000)

# Add custom callback for constraint loss
losshistory, train_state = model.train(epochs=10000, callbacks=[dde.callbacks.LossTerm(custom_loss)])

# Plot and analyze results (optional)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.027337 s



c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


TypeError: Argument `fetch` = None has invalid type "NoneType". Cannot be None